# Neural Network for Happiness Score Estimation

In [7]:
#importing dependencies

import torch
import torchvision
import torchvision.transforms as transforms

In [8]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('2', '3', '4',
           '5', '6', '7', '8')

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [10]:
# defining the Convolution Neural Network

import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [11]:
# loss Function Generation

import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [12]:
# Training the Network

for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 2.236
[1,  4000] loss: 1.950
[1,  6000] loss: 1.732
[1,  8000] loss: 1.626
[1, 10000] loss: 1.553
[1, 12000] loss: 1.493
[2,  2000] loss: 1.443
[2,  4000] loss: 1.391
[2,  6000] loss: 1.364
[2,  8000] loss: 1.342
[2, 10000] loss: 1.309
[2, 12000] loss: 1.277
Finished Training


Here we have to save the trained Neural Network to device, which is CUDA in our case.

In [15]:
PATH ='C:/Users/reddy/Documents/GitHub/Datathon/Machine Learning/happinesstraining_net.pth'
torch.save(net.state_dict(), PATH)

The next steps after training are to test the data given certain types that do not currently have Happiness scores. The way we do this is the following: 

In [16]:
'''
dataiter = iter(testloader)
images, labels = next(dataiter)

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join(f'{classes[labels[j]]:5s}' for j in range(4)))
'''

"\ndataiter = iter(testloader)\nimages, labels = next(dataiter)\n\n# print images\nimshow(torchvision.utils.make_grid(images))\nprint('GroundTruth: ', ' '.join(f'{classes[labels[j]]:5s}' for j in range(4)))\n"

The reason I surrounded code in comments is because we currently do not have the test data (countries with GDP, Population, Life Expectancy, and Size of Country. When given more time, we can web-scape to get more data and put it into this.